# Tensor product of chain complexes

## Introduction

One knows that the homology groups of a cartesian product of two spaces $K$ and $L$ may be obtained by considering a chain complex derived from respective chain complexes of $K$ and $L$ by taking their *tensor product*.
But this is only one of the numerous uses  of tensor products of chain complexes in algebraic topology, so the ``Kenzo`` software provides the handling of such an important tool.

Let us recall that chain complexes are free $\mathbb{Z}$-modules with distinguished basis. A tensor product of chain complexes 
is itself a free $\mathbb{Z}$-module with a natural basis formed by the tensor product of the generators of the chain complex factors. The program conforms to that rule.

## Tensor product of generators and combinations

An elementary tensor product of two generators is represented, in the software, by a structured list in which we may find 4 items for the description of the two generators together with their respective degree. The internal representation of $gnrt1 \otimes gnrt2$ has the form:
```lisp
(:tnpr (degr1 . gnrt1) .  (degr2 . gnrt2))
```
where,
1. *degr1* is an integer, the degree of the generator *gnrt1*.
2. *gnrt1* is the first generator of the pair.
3. *degr2* is an integer, the degree of the generator *gnrt2*.
4. *gnrt2* is  the second generator of the pair.
To construct such an object, one may use the macro ``tnpr``. The corresponding type is ``TNPR``. The printing method prints the  tensor product under the form:
>``<TnPr`` *gnrt1 gnrt2*``>`` or ``<TnPr`` *degr1 gnrt1 degr2 gnrt2*``>``

according to the boolean value, ``NIL`` (default) or ``T``  of the system variable  ``*tnpr-with-degrees*`` (see the examples).

### Simple functions for the tensor product


``(tnpr degr1 gnrt1 degr2 gnrt2)`` *[Macro]*
> Build a tensor product $gnrt1 \otimes gnrt2$.

``(tnpr-p object)`` *[Predicate]*
> Test if *object* is of type ``TNPR``.

``(degr1 tnpr)`` *[Macro]*
> Select the degree of the first generator in the tensor product *tnpr*.

``(gnrt1 tnpr)`` *[Macro]*
> Select the first generator from the object *tnpr*.

``(degr2 tnpr)`` *[Macro]*
> Select the degree of the second generator in the tensor product *tnpr*.

``(gnrt2 tnpr)`` *[Macro]*
> Select the second generator from the object *tnpr*.

``(2cmbn-tnpr cmbn1 cmbn2)`` *[Function]*
> Create, from two combinations *cmbn1* and *cmbn2* with respective degree *degr1* and *degr2*, a combination of degree $dgr1 + dgr2$  by applying the tensorial distributive law on the two sums of terms of the combinations.

#### Example

In [1]:
(tnpr 1 'a 2 'b)

<TnPr A B>

In [2]:
(tnpr-p *)

T

In [3]:
(tnpr-p (cmbn 0 1 'a 2 'b))

NIL

In [4]:
(setf *tnpr-with-degrees* t)

T

In [5]:
(2cmbn-tnpr (cmbn 2 3 'a 4 'b -5 'c) (cmbn 3 4 'x -3 'y 2 'z))


----------------------------------------------------------------------{CMBN 5}
<12 * <TnPr 2 A 3 X>>
<-9 * <TnPr 2 A 3 Y>>
<6 * <TnPr 2 A 3 Z>>
<16 * <TnPr 2 B 3 X>>
<-12 * <TnPr 2 B 3 Y>>
<8 * <TnPr 2 B 3 Z>>
<-20 * <TnPr 2 C 3 X>>
<15 * <TnPr 2 C 3 Y>>
<-10 * <TnPr 2 C 3 Z>>
------------------------------------------------------------------------------


In [6]:
(setf *tnpr-with-degrees* nil)

NIL

In [7]:
**


----------------------------------------------------------------------{CMBN 5}
<12 * <TnPr A X>>
<-9 * <TnPr A Y>>
<6 * <TnPr A Z>>
<16 * <TnPr B X>>
<-12 * <TnPr B Y>>
<8 * <TnPr B Z>>
<-20 * <TnPr C X>>
<15 * <TnPr C Y>>
<-10 * <TnPr C Z>>
------------------------------------------------------------------------------


## Tensor product of chain complexes

The software implements the tensor product of chain complexes according to the classical following definition. Let $C$ and $C'$ two chain complexes. The tensor product $C\otimes C'$ is the chain complex $D$ such that:

$$D_p=\bigoplus_{m+n=p}C_m\otimes C_n',$$

$C_m\otimes C_n'$ being the tensor product of the two modules $C_m$ and $C_n'$. A basis for $D_p$ is the union of the basis of $C_m\otimes C_n'$, with $m+n=p$.

The boundary operator $d^{\otimes}$ is defined, according to the Koszul rule, by:

$$d^{\otimes}(c_m\otimes c_n')=d(c_m)\otimes c_n' +(-1)^m c_m\otimes d'(c_n'),$$

with $c_m\in C_m,\ c_n'\in C_n'$ and $d$, $d'$ being the respective boundary operators
of $C$ and $C'$.

In the software, this is realized by the function ``tnsr-prdc``. 

``(tnsr-prdc chcm1 chcm2)`` *[Method]*
> Build a chain complex, tensor product of the two chain complexes *chcm1* and *chcm2*. The elements
of this new chain complex are integer combinations of generators of *TNPR* type. The creation of this
new chain complex is done by a call to the function *build-chcm* with actual parameters defined from the constituting elements  of *chcm1* and *chcm2*, according to the mathematical definitions above. If both arguments are *effective*, the function constructs an *effective* chain complex. On the other hand, if at least one of the chain complex is *locally effective*, the tensor product is also *locally effective*. In fact, the construction is correct only if both chain complexes are null in negative degrees, otherwise the result is undefined.

#### Examples

Let us take the standard 2-simplex, $\Delta^2$ and let us build $C_*(\Delta^2) \otimes C_*(\Delta^2)$. To build the corresponding chain complex, we use the function ``cdelta``, defined in a previous chapter.

In [8]:
(cat-init)


---done---

NIL

In [9]:
(defun cdelta (dmns)
    (build-chcm
     :cmpr #'l-cmpr
     :basis #'(lambda (n)
                      (mapcar #'dlop-int-ext (funcall (delta-n-basis dmns) n)))
     :bsgn '(0)
     :intr-dffr
         #'(lambda (degr gmsm)
            (make-cmbn
             :degr (1- degr)
             :list (do ((rslt +empty-list+
                              (cons (cons sign
                                          (append
                                           (subseq gmsm 0 nark)
                                           (subseq gmsm (1+ nark))))
                                    rslt))
                        (sign 1 (- sign))
                        (nark 0 (1+ nark)))
                       ((> nark degr) rslt))))
     :strt :gnrt
     :orgn `(Effective version of C_* delta ,dmns)))

CDELTA

In [10]:
(def triangle (cdelta 2))

[K1 Chain-Complex]

In [11]:
(basis triangle 1)

((0 1) (0 2) (1 2))

In [12]:
(def tpr-triangles (tnsr-prdc triangle triangle))

[K3 Chain-Complex]

Let us inspect some basis of this newly created chain complex.

In [13]:
(basis tpr-triangles 0)

(<TnPr (0) (0)> <TnPr (0) (1)> <TnPr (0) (2)> <TnPr (1) (0)> <TnPr (1) (1)> <TnPr (1) (2)> <TnPr (2) (0)> <TnPr (2) (1)> <TnPr (2) (2)>)

In [14]:
(basis tpr-triangles 1)

(<TnPr (0) (0 1)> <TnPr (0) (0 2)> <TnPr (0) (1 2)> <TnPr (1) (0 1)> <TnPr (1) (0 2)> <TnPr (1) (1 2)> <TnPr (2) (0 1)> <TnPr (2) (0 2)> <TnPr (2) (1 2)> <TnPr (0 1) (0)> ...)

In [15]:
(basis tpr-triangles 2)

(<TnPr (0) (0 1 2)> <TnPr (1) (0 1 2)> <TnPr (2) (0 1 2)> <TnPr (0 1) (0 1)> <TnPr (0 1) (0 2)> <TnPr (0 1) (1 2)> <TnPr (0 2) (0 1)> <TnPr (0 2) (0 2)> <TnPr (0 2) (1 2)> <TnPr (1 2) (0 1)> ...)

In [16]:
(basis tpr-triangles 3)

(<TnPr (0 1) (0 1 2)> <TnPr (0 2) (0 1 2)> <TnPr (1 2) (0 1 2)> <TnPr (0 1 2) (0 1)> <TnPr (0 1 2) (0 2)> <TnPr (0 1 2) (1 2)>)

In [17]:
(basis tpr-triangles 4)

(<TnPr (0 1 2) (0 1 2)>)

Let us consider now the  chain complex, $ccn$, that we used in the chain complex chapter. The basis in any degree are decades produced by the function ``<a-b<``. We build $ccn\otimes ccn$ and we verify the fundamental property of the associated boundary operator:

$$d^\otimes \circ d^\otimes=0.$$

In [18]:
(def ccn-boundary #'(lambda (dgr gnr)
     (if (evenp (+ dgr gnr))
         (cmbn (1- dgr) 1 (- gnr 10))
         (cmbn (1- dgr)))))

#<FUNCTION (LAMBDA (DGR GNR) :IN "/root/cl-jupyter-master/cl-jupyter.lisp") {1005ACB71B}>

In [19]:
(def ccn (build-chcm :cmpr #'f-cmpr
                     :basis #'(lambda (n) (<a-b< (* 10 n) (* 10 (1+ n))))
                     :intr-dffr  ccn-boundary
                     :strt :gnrt
                     :orgn '(ccn) ))

[K5 Chain-Complex]

In [20]:
(basis ccn 3)

(30 31 32 33 34 35 36 37 38 39)

In [21]:
(def tpr-ccn-ccn (tnsr-prdc ccn ccn))

[K7 Chain-Complex]

In [22]:
(def comb2 (cmbn 2 1 21 5 25 9 29))


----------------------------------------------------------------------{CMBN 2}
<1 * 21>
<5 * 25>
<9 * 29>
------------------------------------------------------------------------------


In [23]:
(def comb3 (cmbn 3 2 32 3 33 -4 34 -6 36))


----------------------------------------------------------------------{CMBN 3}
<2 * 32>
<3 * 33>
<-4 * 34>
<-6 * 36>
------------------------------------------------------------------------------


In [24]:
(def tcmb (2cmbn-tnpr comb2 comb3))


----------------------------------------------------------------------{CMBN 5}
<2 * <TnPr 21 32>>
<3 * <TnPr 21 33>>
<-4 * <TnPr 21 34>>
<-6 * <TnPr 21 36>>
<10 * <TnPr 25 32>>
<15 * <TnPr 25 33>>
<-20 * <TnPr 25 34>>
<-30 * <TnPr 25 36>>
<18 * <TnPr 29 32>>
<27 * <TnPr 29 33>>
... ...
------------------------------------------------------------------------------


In [25]:
(? tpr-ccn-ccn *)


----------------------------------------------------------------------{CMBN 4}
<3 * <TnPr 21 23>>
<15 * <TnPr 25 23>>
<27 * <TnPr 29 23>>
------------------------------------------------------------------------------


In [26]:
(? tpr-ccn-ccn *)


----------------------------------------------------------------------{CMBN 3}
------------------------------------------------------------------------------


## Tensor product of morphisms, reductions, homotopy equivalences.

Let $f:A_1 \longrightarrow A_2$ and $g:B_1 \longrightarrow B_2$ two morphisms between 
two chain complexes. The tensor product $f \otimes g$ is the morphism

$$f\otimes g : A_1\otimes B_1 \longrightarrow A_2\otimes B_2$$ 

defined by the following formula respecting the Koszul rule:

$$ (f\otimes g) (a_1 \otimes b_1)= (-1)^{deg(g)*deg(a1)}f(a_1) \otimes g(b_1).$$

The method ``tnsr-prdc`` already defined for chain complexes, may be also used for that purpose. 

``(tnsr-prdc mrph1 mrph2)`` *[Method]*
> Return the morphism, tensor product of the two morphisms *mrph1* and *mrph2*. The source
and target of this new morphism are respectively the tensor product of the chain complexes source
and target of *mrph1* and *mrph2*. The degree is the sum of the degrees of the parameters
and the lisp function (the ``:pure`` keyword argument of the function ``build-mrph``) conforms to the mathematical definition above. The strategy is by generator, i.e. the morphism is designed to work with 2 arguments: a degree and a generator which must be an object of type ``TNPR``.

``(tnsr-prdc rdct1 rdct2)`` *[Method]*
> Return the reduction,  tensor product of the two reductions *rdct1* and *rdct2*. The algorithm consists essentially in defining
$$f=f1 \otimes f2,\, g=g1 \otimes g2,\, h= h1 \otimes (g2 \circ g2) + Id_{tcc1} \otimes h2,$$
where $Id_{tcc1}$ is the identity morphism in the top chain complex of the reduction *rdct1*.
The returned reduction is then built by a call to the method ``build-rdct``. This defines completely the chain complexes involved in the reduction.

``(tnsr-prdc hmeq1 hmeq2)`` *[Method]*
> Return the homotopy equivalence, tensor product of the two homotopy equivalence *heqm1* and *heqm2*. 
This is a homotopy equivalence where the new reductions are the tensor products of the arguments reductions (See the lisp definition just below).

### Searching homology for tensor products

The comment list of a tensor product of two chain complexes has the form 
``(TNSR-PRDC`` *chcm1 chcm2*``)``. The ``search-efhm`` method applied to a tensor product, looks for the value of the respective ``efhm`` slots of the chain complexes *chcm1* and *chcm2*, (i.e. two homotopy equivalences) or tries to settle these slots if they are still unbound. Then it builds the tensor product of both homotopy equivalences, as shown in the following lisp listing. At its turn,
the resulting homotopy equivalence will become the value of the {\tt efhm} slot of the initial tensor product chain complex.

```lisp
(defmethod SEARCH-EFHM (chcm (orgn (eql 'tnsr-prdc)))
  (declare (type chain-complex chcm))
  (the homotopy-equivalence
       (tnsr-prdc (efhm (second (orgn chcm)))
                  (efhm (third (orgn chcm))))))

(defmethod TNSR-PRDC ((hmeq1 homotopy-equivalence)
		      (hmeq2 homotopy-equivalence))
  (the homotopy-equivalence
     (build-hmeq
        :lrdct (tnsr-prdc (lrdct hmeq1) (lrdct hmeq2))
        :rrdct (tnsr-prdc (rrdct hmeq1) (rrdct hmeq2))
        :orgn `(tnsr-prdc ,hmeq1 ,hmeq2))))
```

#### Lisp file concerned in this chapter

``tensor-products.lisp``, ``searching-homology.lisp``.